# Procesado de los datos

In [ ]:
import pandas as pd
import geohash2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

### Obtener df de geohashes únicos, latitud y longitud de cada geohash y asignar variables

In [ ]:
def process_dataframe(df):
    """
    Procesa un DataFrame realizando las siguientes operaciones.
    Pasos:
    1. Obtiene valores únicos de geohashes de una columna específica
    2. Decodifica geohashes y agrega columnas de Latitud y Longitud
    3. Asigna a cada geohash las bandas de frecuencia y CGI que tengan las conexiones para ese geohash

    Argumentos:
    df (pandas.DataFrame): El DataFrame inicial.

    Returns:
    pandas.DataFrame: El DataFrame procesado final.
    """

    columna_geohash = 'Geohash'
    valores_unicos = df[columna_geohash].drop_duplicates()
    geohash_df = pd.DataFrame({'Geohash': valores_unicos})

    geohash_df['geometry'] = geohash_df['Geohash'].apply(lambda x: geohash2.decode_exactly(x)[:2])
    geohash_df[['Latitud', 'Longitud']] = geohash_df['geometry'].apply(lambda x: pd.Series([str(x[0]), str(x[1])]))
    geohash_df.drop('geometry', axis=1, inplace=True)

    geohash_df['BAND_FREQ'] = None
    band_freq_unique = df.groupby('Geohash')['BAND_FREQ'].unique()
    for index, row in geohash_df.iterrows():
        geohash = row['Geohash']
        if geohash in band_freq_unique:
            geohash_df.at[index, 'BAND_FREQ'] = band_freq_unique[geohash].tolist()
    new_rows = []
    
    for index, row in geohash_df.iterrows():
        geohash = row['Geohash']
        latitud = row['Latitud']
        longitud = row['Longitud']
        band_freq_list = row['BAND_FREQ']
        for band_freq in band_freq_list:
            new_rows.append({'Geohash': geohash, 'Latitud': latitud, 'Longitud': longitud, 'BAND_FREQ': band_freq})
    geohash_df = pd.DataFrame(new_rows)
    
    ci_unicos = df.groupby(['Geohash', 'BAND_FREQ'])['CGI'].apply(lambda x: list(set(x)))
    geohash_df['CGI'] = None
    for index, row in geohash_df.iterrows():
        geohash = row['Geohash']
        band_freq = row['BAND_FREQ']
        if (geohash, band_freq) in ci_unicos.index:
            cell_ids = ci_unicos.loc[(geohash, band_freq)]
            geohash_df.at[index, 'CGI'] = cell_ids
            
    new_rows = []
    for index, row in geohash_df.iterrows():
        geohash = row['Geohash']
        latitud = row['Latitud']
        longitud = row['Longitud']
        band_freq = row['BAND_FREQ']
        cgi_list = row['CGI']
        for cgi in cgi_list:
            new_rows.append({'Geohash': geohash, 'Latitud': latitud, 'Longitud': longitud,'BAND_FREQ': band_freq,'CGI': cgi})
    geohash_df = pd.DataFrame(new_rows)

    return geohash_df

### Df para RSRP y RSRQ, calcular media de cada variable y número de eventos

In [ ]:
def process_dataframe_rsrp_rsrq(df_copia, df_valores):
    """
    Procesa un DataFrame para calcular métricas RSRP y RSRQ.
    Pasos:
    1. Crea dos copias del DataFrame, una será para la RSRP y otra para la RSRQ
    2. En el caso de la RSRP, calcula la RSRP_media y el número de eventos buscando coincidencias en el DataFrame de 
       las medidas
    3. En el caso de la RSRQ, asigna franja horarias y posteriormente calcula la RSRP_media y el número de eventos buscando 
       coincidencias en el DataFrame de las medida
    
    Argumentos:
    df_copia (pandas.DataFrame): El DataFrame que se va a copiar para rsrp y rsrq.
    df_valores (pandas.DataFrame): El DataFrame del cual se extraen los valores de RSRP y RSRQ.

    Returns:
    rsrp_df (pandas.DataFrame): El DataFrame que contiene los datos con la RSRP calculada.
    rsrq_df (pandas.DataFrame): El DataFrame que contiene los datos con la RSRQ calculada.
    """

    rsrp_df = df_copia.copy()
    rsrq_df = df_copia.copy()

    rsrp_df['RSRP_media'] = None
    rsrp_df['eventos'] = None
    for index, row in rsrp_df.iterrows():
        geohash = row['Geohash']
        band_freq = row['BAND_FREQ']
        cgi = row['CGI']
        match_rsrp = df_valores[(df_valores['Geohash'] == geohash) & (df_valores['BAND_FREQ'] == band_freq) & (df_valores['CGI'] == cgi)]
        if not match_rsrp.empty:
            rsrp_df.at[index, 'RSRP_media'] = np.mean(match_rsrp['RSRP'])
            rsrp_df.at[index, 'eventos'] = len(match_rsrp)

    rsrq_df['Franja_horaria'] = None
    for index, row in rsrq_df.iterrows():
        geohash = row['Geohash']
        band_freq = row['BAND_FREQ']
        cgi = row['CGI']
        match_franja = df_valores[(df_valores['Geohash'] == geohash) & (df_valores['BAND_FREQ'] == band_freq) & (df_valores['CGI'] == cgi)]
        if not match_franja.empty:
            franjas = list(set(match_franja['Franja_horaria']))
            rsrq_df.at[index, 'Franja_horaria'] = franjas

    rsrq_df['Franja_horaria'] = rsrq_df['Franja_horaria'].apply(lambda x: [x] if isinstance(x, str) else x)
    rsrq_df = rsrq_df.explode('Franja_horaria')

    rsrq_df['RSRQ_media'] = None
    rsrq_df['eventos'] = None
    for index, row in rsrq_df.iterrows():
        geohash = row['Geohash']
        band_freq = row['BAND_FREQ']
        cgi = row['CGI']
        franja = row['Franja_horaria']
        match_rsrq = df_valores[(df_valores['Geohash'] == geohash) & (df_valores['BAND_FREQ'] == band_freq) & (df_valores['CGI'] == cgi) & (df_valores['Franja_horaria'] == franja)]
        if not match_rsrq.empty:
            rsrq_df.at[index, 'RSRQ_media'] = np.mean(match_rsrq['RSRQ'])
            rsrq_df.at[index, 'eventos'] = len(match_rsrq)

    rsrp_df = rsrp_df[['Geohash', 'Latitud', 'Longitud', 'BAND_FREQ', 'CGI', 'RSRP_media', 'eventos']]
    rsrq_df = rsrq_df[['Geohash', 'Latitud', 'Longitud', 'BAND_FREQ', 'CGI', 'Franja_horaria', 'RSRQ_media', 'eventos']]

    return rsrp_df, rsrq_df

In [ ]:
def filtrar_coincidencias(df1, df2, df3, df4, columns_to_match1 = None, columns_to_match2 = None):
    """
    Compara dos DataFrames con el fin de quedarse únicamente con aquellos datos que estén presentes en ambos Dataframes.

    Argumentos:
    df1 (pandas.DataFrame): El DataFrame procesado que contiene los datos de RSRP para las medidas del drive test
    df2 (pandas.DataFrame): El DataFrame procesado que contiene los datos de RSRP para las medidas crowdsourced
    df3 (pandas.DataFrame): El DataFrame procesado que contiene los datos de RSRQ para las medidas del drive test
    df4 (pandas.DataFrame): El DataFrame procesado que contiene los datos de RSRQ para las medidas crowdsourced

    Returns:
    filtered_df1 (pandas.DataFrame): El DataFrame filtrado que contiene los datos de RSRP para las medidas del drive test que 
                                     estén presentes en el conjunto de datos crowdsourced
    filtered_df2 (pandas.DataFrame): El DataFrame filtrado que contiene los datos de RSRP para las medidas crowdsourced que 
                                     estén presentes en el conjunto de datos del drive test
    filtered_df3 (pandas.DataFrame): El DataFrame filtrado que contiene los datos de RSRQ para las medidas del drive test que 
                                     estén presentes en el conjunto de datos crowdsourced
    filtered_df4 (pandas.DataFrame): El DataFrame procesado que contiene los datos de RSRQ para las medidas crowdsourced que 
                                     estén presentes en el conjunto de datos del drive test
    """
    columns_to_match1 = ['Geohash', 'BAND_FREQ', 'CGI']
    columns_to_match2 = ['Geohash', 'BAND_FREQ', 'CGI', 'Franja_horaria']
    def filtrar(df_a, df_b, cols):
        filtered_a = pd.DataFrame(columns=df_a.columns)
        filtered_b = pd.DataFrame(columns=df_b.columns)
        
        for _, row in df_a.iterrows():
            coincidencia = df_b
            for col in cols:
                coincidencia = coincidencia[coincidencia[col] == row[col]]
            if not coincidencia.empty:
                filtered_a = pd.concat([filtered_a, row.to_frame().T])
        
        for _, row in df_b.iterrows():
            coincidencia = df_a
            for col in cols:
                coincidencia = coincidencia[coincidencia[col] == row[col]]
            if not coincidencia.empty:
                filtered_b = pd.concat([filtered_b, row.to_frame().T])
        
        filtered_a.reset_index(drop=True, inplace=True)
        filtered_b.reset_index(drop=True, inplace=True)
        
        return filtered_a, filtered_b
    
    filtered_df1, filtered_df2 = filtrar(df1, df2, columns_to_match1)
    filtered_df3, filtered_df4 = filtrar(df3, df4, columns_to_match2)
    
    return filtered_df1, filtered_df2, filtered_df3, filtered_df4